In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gzip
from six.moves import cPickle
import sys    

In [2]:
# selecting parameters
# number of convolutional filters to use
nb_filters = 64 

# filter size    
filter_rows = 1
filter_cols = 2 

# convolution kernel size
nb_conv = 3

# subsampling size 
subsample_rows = 1 
subsample_cols_a = 2    
subsample_cols_b = 1

# batch size 
batch_size = 50

# dimension    
latent_dim = 3
intermediate_dim = 128

# epsilon values 
mean=0.
epsilon_std = 1.0      

# dropout 
drop = 0

# epoch, iteration sizes  
nb_epoch = 1  

# saving models: start & end
nb_start = 0
nb_end = 400       
total_epochs = (nb_end-nb_start)*nb_epoch  
print "Total of %i epochs are set to run" % total_epochs  

Total of 400 epochs are set to run


In [3]:
import os, sys
path_data = './X_110000.pkl.gz' 
path_1 = "./fig"
path_2 = "./imgs"
path_3 = "./hist"
path_4 = "./model"
if not os.path.exists(path_1):
    os.mkdir(path_1, 0755);
if not os.path.exists(path_2):
    os.mkdir(path_2, 0755);
if not os.path.exists(path_3):
    os.mkdir(path_3, 0755);
if not os.path.exists(path_4):
    os.mkdir(path_4, 0755);   
print "directories creation compledted or if already exist - then checked"   

directories creation compledted or if already exist - then checked


In [4]:
with gzip.open(path_data, 'rb') as f3:    
    (x_train_raw, y_train_raw), (x_test_raw,y_test_raw), (x_pred_raw,y_pred_raw) = cPickle.load(f3)  


In [6]:
imag_rows = x_train_raw.shape[0]/len(x_train_raw)
imag_cols = x_train_raw.shape[1]
imag_chns = 1
print imag_rows, imag_cols

1 784


In [49]:
from neon.layers.layer import Layer
class Lambda(Layer):
    " A layer that multiples the input by two. "

    # constructor and initialize buffers
    def __init__(self, name=None):
        super(Lambda, self).__init__(name)

    # configure the layer input and output shapes
    def configure(self, in_obj):
        super(Lambda, self).configure(in_obj)
        self.out_shape = self.in_shape
        return self

    # compute the fprop
    def fprop(self, inputs, inference=False):
        self.outputs = inputs
        self.outputs[:] = self.sampling(inputs)
        return self.outputs
    
    #calculate z_mean
    def sampling(self, inputs):
        z_mean = z_log_var = inputs
        epsilon = np.random.normal(0, 1.0, self.out_shape)
        return z_mean + np.exp(z_log_var)*epsilon

    # backprop the gradients
    def bprop(self, error):
        error[:] = error
        return error

In [42]:
import logging
from neon.callbacks.callbacks import Callbacks
from neon.data import ArrayIterator
from neon.initializers import Gaussian
from neon.layers import GeneralizedCost, Affine, Reshape
from neon.models import Model
from neon.optimizers import GradientDescentMomentum
from neon.transforms import Rectlin, Logistic, CrossEntropyBinary, Misclassification, MeanSquared, Accuracy,Softmax

from neon.layers import Conv, Affine, Pooling, Deconv
from neon.initializers import GlorotUniform

In [43]:
init = GlorotUniform()
activation = Rectlin()

In [44]:
layers = [Conv(fshape=(1, 1, 2), init=init, activation=activation, padding={"pad_h": 1}),
         Conv(fshape=(64, 1, 2), init=init, strides={"str_w":1, "str_h":2},
              activation=activation, padding={"pad_h":1}),
         Conv(fshape=(64, 3, 3), init=init, strides={"str_w":1, "str_h":1},
              activation=activation, padding={"pad_h":2, "pad_w":2}),
         Conv(fshape=(64, 3, 3), init=init, strides={"str_w":1, "str_h":1},
              activation=activation, padding={"pad_h":2, "pad_w":2}),
         Reshape(64*28*28),
         Lambda(),
          Affine(nout=128, init=init, activation=activation),
          Affine(nout=64*28*14, init=init, activation=activation),
          Reshape(64*28*14),
          Deconv(fshape=())
         ]

In [56]:
def sampling(inputs):
    z_mean = z_log_var = inputs
    print z_mean
    print z_log_var
    epsilon = np.random.normal(0, 1.0, (1, 3))
    return z_mean + np.exp(z_log_var)*epsilon

In [60]:
a = np.array([0.3, 0.2, 0.4])
b = a
b[:] = sampling(a)
print a
print b


[ 0.3  0.2  0.4]
[ 0.3  0.2  0.4]
[-1.26155414 -0.11828013 -0.74879457]
[-1.26155414 -0.11828013 -0.74879457]
